In [28]:
import pandas as pd

In [29]:
googleSheetId = '1BGNomIEkIgiPaPbMpeCArVCpERKK9TGHYFyT4Mt9grk'
worksheetName = 'sheet1'
URL = f'https://docs.google.com/spreadsheets/d/{googleSheetId}/gviz/tq?tqx=out:csv&sheet={worksheetName}'
inscricoes = pd.read_csv(URL)

In [30]:
googleSheetId = '1bNqrMwDAPVNG8YxzEJt6sZNmtO9xEdHjIvDUw8c_oyY'
worksheetName = 'sheet1'
URL = f'https://docs.google.com/spreadsheets/d/{googleSheetId}/gviz/tq?tqx=out:csv&sheet={worksheetName}'
cancelamentos = pd.read_csv(URL)

In [31]:
inscricoes.Timestamp = pd.to_datetime(inscricoes.Timestamp)
cancelamentos.Timestamp = pd.to_datetime(cancelamentos.Timestamp)

In [32]:
ultimo_cancelamento = cancelamentos.sort_values('Timestamp', ascending=False).groupby('E-mail').head(1)

In [33]:
compilado = inscricoes.merge(ultimo_cancelamento, 
                             how='left', 
                             left_on='Informe seu e-mail institucional', 
                             right_on='E-mail',
                             suffixes=['_incricao', '_cancelamento'])

In [34]:
compilado

,Timestamp_incricao,Informe seu e-mail institucional,Sobre quais acidentes deseja ser notificado?,As notificações devem ter qual abrangência geográfica?,Selecione a UF,Selecione a UORG,Timestamp_cancelamento,E-mail
0,2022-07-19 11:40:46,joao.reis@economia.gov.br,Acidentes graves e fatais,Unidade da Federação (ex.: MG),AC,NaN,2022-07-22 09:23:43,joao.reis@economia.gov.br
1,2022-07-19 11:41:51,joao.reis@economia.gov.br,Acidentes fatais,Unidade Organizacional (ex.: Gerência de Vargi...,NaN,AM - 025901000 - GRTE/TABATINGA/AM,2022-07-22 09:23:43,joao.reis@economia.gov.br
2,2022-07-22 09:28:07,joao.reis@economia.gov.br,Acidentes graves e fatais,Unidade da Federação (ex.: MG),DF,NaN,2022-07-22 09:23:43,joao.reis@economia.gov.br
3,2022-07-19 11:40:46,joao.reis@economia.gov.br,Acidentes graves e fatais,Unidade da Federação (ex.: MG),AL,NaN,2022-07-22 09:23:43,joao.reis@economia.gov.br


In [36]:
compilado['intervalo'] = compilado.Timestamp_cancelamento - compilado.Timestamp_incricao

In [40]:
compilado

,Timestamp_incricao,Informe seu e-mail institucional,Sobre quais acidentes deseja ser notificado?,As notificações devem ter qual abrangência geográfica?,Selecione a UF,Selecione a UORG,Timestamp_cancelamento,E-mail,intervalo
0,2022-07-19 11:40:46,joao.reis@economia.gov.br,Acidentes graves e fatais,Unidade da Federação (ex.: MG),AC,NaN,2022-07-22 09:23:43,joao.reis@economia.gov.br,2 days 21:42:57
1,2022-07-19 11:41:51,joao.reis@economia.gov.br,Acidentes fatais,Unidade Organizacional (ex.: Gerência de Vargi...,NaN,AM - 025901000 - GRTE/TABATINGA/AM,2022-07-22 09:23:43,joao.reis@economia.gov.br,2 days 21:41:52
2,2022-07-22 09:28:07,joao.reis@economia.gov.br,Acidentes graves e fatais,Unidade da Federação (ex.: MG),DF,NaN,2022-07-22 09:23:43,joao.reis@economia.gov.br,-1 days +23:55:36
3,2022-07-19 11:40:46,joao.reis@economia.gov.br,Acidentes graves e fatais,Unidade da Federação (ex.: MG),AL,NaN,2022-07-22 09:23:43,joao.reis@economia.gov.br,2 days 21:42:57


In [41]:
from datetime import timedelta
compilado['vigente'] = compilado.intervalo < timedelta(microseconds=0)

In [46]:
compilado[compilado.vigente == True]

,Timestamp_incricao,Informe seu e-mail institucional,Sobre quais acidentes deseja ser notificado?,As notificações devem ter qual abrangência geográfica?,Selecione a UF,Selecione a UORG,Timestamp_cancelamento,E-mail,intervalo,vigente
2,2022-07-22 09:28:07,joao.reis@economia.gov.br,Acidentes graves e fatais,Unidade da Federação (ex.: MG),DF,NaN,2022-07-22 09:23:43,joao.reis@economia.gov.br,-1 days +23:55:36,True
